# auxiliar

In [1]:
import pandas as pd

In [2]:
!python -m spacy download es_core_news_sm

2023-07-05 22:23:25.438798: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-05 22:23:26.789274: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 66.9 MB/s eta 0:00:0000:010:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [ ]:
%pip install pycorrector
%pip install unidecode
%pip install pyspellchecker

In [4]:
tipificacion=pd.read_excel("/content/PRIORIZAR-1/data/retipi.xlsx").drop_duplicates()

tipificacion=tipificacion.reset_index()

tipificacion = tipificacion[tipificacion['correccion'] != 'conceptos facturados']
tipificacion


,index,NUMERO_PQR,DESCRIPCION_PQR_x,correccion
0,0,MDM-PQR-35424371,SG//LT: 6016133556 cuenta 12054410753// client...,INCREMENTO DE TARIFA
1,1,MDM-PQR-35424385,SG//LT: 6017257398 cuenta 12054364769// client...,INCREMENTO DE TARIFA
2,2,MDM-PQR-35424468,SG//LT: 6013835268 cuenta 12053820540// client...,INCREMENTO DE TARIFA
3,3,MDM-PQR-35425107,Línea: 6016456888 Nombre de quien se comunica:...,SVAS
4,4,MDM-PQR-35426399,CLIENTE DESEA CONOCER LA FACTURA GENERADA,otros
...,...,...,...,...
1521,1623,MDM-PQR-37429362,EXPLICACION FACTURACION,conceptos facturados
1522,1624,MDM-PQR-37429363,explicacion de factura,conceptos facturados
1523,1625,MDM-PQR-37429364,WHATSAPP// EXPLICACI̒N DE FACTURA,conceptos facturados
1524,1626,MDM-PQR-37429365,explicacion de la factura,conceptos facturados


In [5]:
tipificacion['correccion'].value_counts()

INCREMENTO DE TARIFA      347
otros                     334
ASEGURAMIENTO             211
conceptos facturados      191
SVAS                      175
BENEFICIO DE RETENCION    154
compensacin               113
Name: correccion, dtype: int64

In [6]:
import pandas as pd

import gc

import seaborn as sns
import plotly.express as px

import numpy as np

import sqlite3
from sqlite3 import Error
import os
import re, string
from nltk.corpus import stopwords
import nltk
import matplotlib.pyplot as plt

import torch
#from transformers import AutoTokenizer, BertForSequenceClassification





nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


import pandas as pd
import re, string


pd.options.display.max_columns = None

#from sentence_transformers import SentenceTransformer, util
#from transformers import *

import spacy

nlp = spacy.load('es_core_news_sm', disable=['tagger', 'parser', 'ner'])



from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import preprocessing

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# aca empieza el modelo

In [7]:
sys.path.append('../librerias')


In [8]:
from clean_text import clean_text

entro2


In [9]:

#declare answers and questions
descripcion=tipificacion.DESCRIPCION_PQR_x	



# Cleaning the questions

clean_descripcion = []
for des in descripcion:
    clean_descripcion.append(clean_text(des))


#clean_descripcion=[k for k in clean_descripcion if 'soporte'  in k]

clean_descripcion  [881]

'  solicita ajuste de la factura ya que reclama por el cobro de reconexion solucion de aseguramiento se cierra caso ya que se evidencia  del dia  reteniendo al usuario justificacion de penalizacion ajuste valor ajustado en front  incluido iva valor real ajuste  incluido iva diferencia  ajuste correcto'

spelling

In [10]:
from spelling import correct_sentence


corrected_text_list = [correct_sentence(text) for text in clean_descripcion]
clean_descripcion=corrected_text_list

hola


In [11]:
clean_descripcion[81]

'loca se penalizar asesor ya que no tiene encanta en el traslado paso de duo a trio por tanto el cobro de la clausula de permanencia en el servicio de tv'

lemma token 

In [12]:
from lemastem import data_preprocessing


clean_descripcion=data_preprocessing(clean_descripcion)
clean_descripcion[3]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


11659


['unicar', 'llego', 'cobro', 'hbo']

In [66]:
merged_data=[]
for i in range(len(clean_descripcion)):
    merged_data.append(" ".join(clean_descripcion[i]))

merged_df=pd.DataFrame(merged_data,columns =['descripcion'])


from unidecode import unidecode

def remove_accents(text):
    return unidecode(text)


# Aplicar la función a la columna 'Texto'
merged_df['descripcion'] = merged_df['descripcion'].apply(remove_accents)


tipificacion=pd.concat([tipificacion, merged_df], axis=1).loc[:, ['descripcion', 'correccion']]

#merged_df = pd.merge(merged_df,leo , on='index', how='inner').loc[:, ['descripcion', 'correccion']]
#############################################




tipificacion = tipificacion[tipificacion['correccion'] != 'SVAS']


tipificacion

,descripcion,correccion
0,solicitar ajuste factura acuerdo cobro reconex...,otros
1,solicitar ajuste factura acuerdo cobro reconex...,otros
2,solicitar ajuste factura acuerdo cobro reconex...,otros
4,conocer factura generado,otros
6,locacion tarifa retencion ofrecer bajo oferta ...,ASEGURAMIENTO
...,...,...
1292,factura incremento solicitar cancelacion ofrec...,INCREMENTO DE TARIFA
1293,consulta aumento factura,INCREMENTO DE TARIFA
1295,llego factura tenia llegar,otros
1296,picar,otros


In [38]:
import pandas as pd
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# Assuming you have a DataFrame called 'data' with 'categories' column
# where categories are labeled as 1, 2, and 3
mapping = {'otros': 0, 'INCREMENTO DE TARIFA': 1, 'ASEGURAMIENTO': 2,  'BENEFICIO DE RETENCION': 3,'compensacin': 4}

# Separate features and target variable
X = tipificacion.drop('correccion', axis=1)
y = tipificacion['correccion'].replace(mapping)

# Oversampling using RandomOverSampler
oversampler = RandomOverSampler(sampling_strategy='auto')
X_resampled, y_resampled = oversampler.fit_resample(X, y)






mapping = {'otros': 0, 'INCREMENTO DE TARIFA': 1, 'ASEGURAMIENTO': 2,  'BENEFICIO DE RETENCION': 3}

crear matriz de numeros 

In [39]:
from tfidf import TFIDF

In [42]:

from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

from joblib import dump


X = X_resampled['descripcion']

y = y_resampled




# train test split (66% train - 33% test)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=123)




X_train,X_test = TFIDF(X_train,X_test)

pca = PCA()

pca.fit(X_train)

# Get the explained variance ratio for each component
explained_variance_ratio = pca.explained_variance_ratio_

# Calculate the cumulative explained variance ratio
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)

# Find the number of components that explain a desired amount of variance
desired_variance = 0.95  # Set the desired variance threshold
n_components = np.argmax(cumulative_variance_ratio >= desired_variance) + 1

# Create a new instance of PCA with the optimal number of components
pca = PCA(n_components=n_components)


X_train_new = pca.fit_transform(X_train)
X_test_new = pca.transform(X_test)

print("train with old features: ",np.array(X_train).shape)
print("train with new features:" ,np.array(X_train_new).shape)

print("test with old features: ",np.array(X_test).shape)
print("test with new features:" ,np.array(X_test_new).shape)





print('Training Data :', X_train.shape)

print('Testing Data : ', X_test.shape)


dump(value=pca, filename="/content/PRIORIZAR/modelo/pca_retipificacion.joblib")



tf-idf with 982 features
train with old features:  (1494, 982)
train with new features: (1494, 339)
test with old features:  (374, 982)
test with new features: (374, 339)
Training Data : (1494, 982)
Testing Data :  (374, 982)


['/content/PRIORIZAR/modelo/pca_retipificacion.joblib']

In [43]:
pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 7.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.1 MB/s eta 0:00:00


In [45]:
import optuna
from optuna.samplers import TPESampler

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score



In [46]:
def objective(trial):
    """
    Objective function to be minimized.
    """
    param = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "num_class": 3,
        "n_estimators": trial.suggest_int("n_estimators", 20, 90),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 5.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 5.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 20, 50),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        'learning_rate': trial.suggest_float("learning_rate", 0.04, 0.1),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    
    gbm = lgb.LGBMClassifier(**param)
    gbm.fit(X_train_new, y_train)
    preds = gbm.predict(X_test_new)
    accuracy = accuracy_score(y_test, preds)
    return accuracy

In [47]:
sampler = TPESampler(seed=1)
study = optuna.create_study(study_name="lightgbm", direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=20)


[I 2023-06-20 22:16:04,486] A new study created in memory with name: lightgbm


[LightGBM] [Warning] feature_fraction is set=0.48805353449026784, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.48805353449026784
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] lambda_l2 is set=0.010007110473405195, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.010007110473405195
[LightGBM] [Warning] bagging_fraction is set=0.4554031568612787, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4554031568612787
[LightGBM] [Warning] lambda_l1 is set=0.879294185249291, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.879294185249291


[I 2023-06-20 22:16:05,525] Trial 0 finished with value: 0.8556149732620321 and parameters: {'n_estimators': 49, 'lambda_l1': 0.879294185249291, 'lambda_l2': 0.010007110473405195, 'num_leaves': 29, 'feature_fraction': 0.48805353449026784, 'bagging_fraction': 0.4554031568612787, 'bagging_freq': 2, 'learning_rate': 0.060733643622582864, 'min_child_samples': 43}. Best is trial 0 with value: 0.8556149732620321.


[LightGBM] [Warning] feature_fraction is set=0.9268704618345672, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9268704618345672
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] lambda_l2 is set=0.7069470359901728, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.7069470359901728
[LightGBM] [Warning] bagging_fraction is set=0.4164325559187557, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4164325559187557
[LightGBM] [Warning] lambda_l1 is set=0.1353297938831623, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1353297938831623


[I 2023-06-20 22:16:07,083] Trial 1 finished with value: 0.8288770053475936 and parameters: {'n_estimators': 58, 'lambda_l1': 0.1353297938831623, 'lambda_l2': 0.7069470359901728, 'num_leaves': 26, 'feature_fraction': 0.9268704618345672, 'bagging_fraction': 0.4164325559187557, 'bagging_freq': 5, 'learning_rate': 0.06503828814202763, 'min_child_samples': 58}. Best is trial 0 with value: 0.8556149732620321.


...

[I 2023-06-20 22:17:02,845] Trial 17 finished with value: 0.8770053475935828 and parameters: {'n_estimators': 78, 'lambda_l1': 0.38130701809028844, 'lambda_l2': 0.05302412848756399, 'num_leaves': 48, 'feature_fraction': 0.7296581552899568, 'bagging_fraction': 0.6812516084133043, 'bagging_freq': 4, 'learning_rate': 0.05572595861633193, 'min_child_samples': 82}. Best is trial 11 with value: 0.893048128342246.


[LightGBM] [Warning] feature_fraction is set=0.624855776826383, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.624855776826383
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] lambda_l2 is set=0.03783786054633798, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.03783786054633798
[LightGBM] [Warning] bagging_fraction is set=0.5924535147206048, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5924535147206048
[LightGBM] [Warning] lambda_l1 is set=0.19081258608856905, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.19081258608856905


[I 2023-06-20 22:17:05,170] Trial 18 finished with value: 0.8850267379679144 and parameters: {'n_estimators': 72, 'lambda_l1': 0.19081258608856905, 'lambda_l2': 0.03783786054633798, 'num_leaves': 45, 'feature_fraction': 0.624855776826383, 'bagging_fraction': 0.5924535147206048, 'bagging_freq': 2, 'learning_rate': 0.048484975587022666, 'min_child_samples': 49}. Best is trial 11 with value: 0.893048128342246.


[LightGBM] [Warning] feature_fraction is set=0.40607903153949193, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.40607903153949193
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] lambda_l2 is set=0.12643531122829316, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.12643531122829316
[LightGBM] [Warning] bagging_fraction is set=0.7524182469699469, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7524182469699469
[LightGBM] [Warning] lambda_l1 is set=1.8506466994518016, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.8506466994518016


[I 2023-06-20 22:17:06,420] Trial 19 finished with value: 0.8636363636363636 and parameters: {'n_estimators': 81, 'lambda_l1': 1.8506466994518016, 'lambda_l2': 0.12643531122829316, 'num_leaves': 39, 'feature_fraction': 0.40607903153949193, 'bagging_fraction': 0.7524182469699469, 'bagging_freq': 4, 'learning_rate': 0.056123860189793956, 'min_child_samples': 99}. Best is trial 11 with value: 0.893048128342246.


In [48]:
print('Best parameters:', study.best_params)

Best parameters: {'n_estimators': 90, 'lambda_l1': 4.891566120846485, 'lambda_l2': 0.010196403754110692, 'num_leaves': 42, 'feature_fraction': 0.8508478559579478, 'bagging_fraction': 0.8054451937682544, 'bagging_freq': 3, 'learning_rate': 0.04089514301599394, 'min_child_samples': 9}


In [49]:
mode = lgb.LGBMClassifier(**study.best_params)
mode.fit(X_train_new, y_train)

[LightGBM] [Warning] lambda_l1 is set=4.891566120846485, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.891566120846485
[LightGBM] [Warning] bagging_fraction is set=0.8054451937682544, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8054451937682544
[LightGBM] [Warning] lambda_l2 is set=0.010196403754110692, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.010196403754110692
[LightGBM] [Warning] feature_fraction is set=0.8508478559579478, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8508478559579478
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3


LGBMClassifier(bagging_fraction=0.8054451937682544, bagging_freq=3,
               feature_fraction=0.8508478559579478, lambda_l1=4.891566120846485,
               lambda_l2=0.010196403754110692,
               learning_rate=0.04089514301599394, min_child_samples=9,
               n_estimators=90, num_leaves=42)

In [50]:
y_pred = mode.predict(X_test_new)


In [51]:
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, confusion_matrix, classification_report

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.88      0.73      0.80        92
           1       0.79      0.91      0.85        94
           2       0.98      0.94      0.96        89
           3       0.94      0.98      0.96        99

    accuracy                           0.89       374
   macro avg       0.90      0.89      0.89       374
weighted avg       0.90      0.89      0.89       374



In [52]:
from joblib import dump


In [53]:
dump(value=mode, filename="/content/PRIORIZAR/modelo/retipificacion.joblib")

['/content/PRIORIZAR/modelo/retipificacion.joblib']